In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
FILES_DIR = 'submissions/'
ensemble_files = [
    {'file': 'ALS.csv', 'score': 0.59129},
    {'file': 'simple_baseline.csv', 'score': 0.86692},
    {'file': 'DT (1).csv', 'score': 0.82562},
    {'file': 'GBT (1).csv', 'score': 0.90029},
    {'file': 'LR (1).csv', 'score': 0.89383},
    {'file': 'RF (1).csv', 'score': 0.89654},
    {'file': 'GBT.csv', 'score': 0.86208},
    {'file': 'DT.csv', 'score': 0.85762},
    {'file': 'RF.csv', 'score': 0.86337},
    {'file': 'LR.csv', 'score': 0.86352},
]

In [18]:
df = pd.read_csv('sample_submission.csv').sort_values('TrackID').reset_index(drop=True)

In [ ]:
scores = []
predictions = []
for ensemble_file in ensemble_files:
    predictions.append(
        pd.read_csv(os.path.join(FILES_DIR, ensemble_file['file'])).sort_values('TrackID')['Predictor'].values
    )
    scores.append(ensemble_file['score'])

In [ ]:
S = np.stack(predictions).T * 2 - 1

In [ ]:
StS_inv = np.linalg.inv(S.T.dot(S))
StX = len(S)*(np.array(scores)*2 - 1)

In [ ]:
a_LS = StS_inv.dot(StX)

In [ ]:
df['EnsembleScore'] = S.dot(np.expand_dims(a_LS, axis=-1))

In [ ]:
df['UserID'] = df['TrackID'].str.split('_').str[0]

In [ ]:
df = df.sort_values('EnsembleScore', ascending=False)

In [ ]:
df = pd.concat([
    df.groupby(['UserID']).head(3).assign(Predictor=1),
    df.groupby(['UserID']).tail(3).assign(Predictor=0)
])[['TrackID', 'Predictor']]

In [ ]:
df.to_csv('ensemble.csv', index=None)

The best public score has improved from **0.90029** to **0.90296**